In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
#!python scripts/preprocessing/process_biodiscover.py \
#    --csv_path ../EPT-14-DL-clean.csv \
#    --out_folder data/processed/ept-14

In [3]:
import pandas as pd
df_csv = pd.read_csv('../EPT-14-DL-clean+1.csv')
print(f"Number of classes: {len(df_csv['Species_DNA'].unique())}")
for label in df_csv['Species_DNA'].unique():
    print(label)

Number of classes: 14
Goera pilosa
Potamophylax rotundipennis
Baetis vernus
Baetis rhodani
Caenis horaria
Limnephilus lunatus
Chaetopteryx villosa
Amphinemura standfussi
Anabolia nervosa
Ephemera danica
Athripsodes cinereus
Mystacides azureus
Halesus radiatus
Nemoura cinerea


In [ ]:
# Create train-test-splits. Splitting all individuals of the 14 species into 5 train-validiation-test splits.
#!python scripts/01_train_test_split.py \
#    --csv_path 'data/processed/ept-14/01_EPT_processed.csv' \
#    --target_col 'Species_DNA' \
#    --group_col 'Specimen ID' \
#    --n_splits 5 \
#    --random_state 42 \
#    --out_folder 'data/processed/ept-14' \
#    --shuffle 'False'

In [5]:
!python scripts/preprocessing/test_dataset_config.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept-14/01_EPT+1_processed_5splits_Species_DNA.csv" \
    --label "Log+1_weights" \
    --fold 0

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
train: found 90869 fpaths, 90869 labels
Examples:
[PosixPath('/tf/notebooks/EPT/0_images/4_G3/4_G3_1/1_4_G3_1_2023_12_14-10-05-17-553.PNG'), PosixPath('/tf/notebooks/EPT/0_images/4_G3/4_G3_1/2_4_G3_1_2023_12_14-10-05-17-563.PNG'), PosixPath('/tf/notebooks/EPT/0_images/4_G3/4_G3_2/1_4_G3_2_2023_12_14-10-05-33-000.PNG'), PosixPath('/tf/notebooks/EPT/0_images/4_G3/4_G3_2/2_4_G3_2_2023_12_14-10-05-33-011.PNG'), PosixPath('/tf/notebooks/EPT/0_images/4_G3/

In [6]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin


True

In [ ]:
# 01_ept-14-biomass was used to determine the learning rate for 'Log_weights' (augmentation: flips-rotate-keep-aspect)
# 02_ept-14-biomass was used to train for 100 epochs on 'Log_weights' (augmentation: flips-rotate-keep-aspect)
# 101_ept-14-biomass was used to determine the learning rate for 'Log_weights' (augmentation: EPT-biomass)
# 102_ept-14-biomass was used to train for 100 epochs on 'Log_weights' (augmentation: EPT-biomass)
# +1_EPT-14-biomass was used to train for 100 epochs on 'Log+1_weights' (augmentation: EPT-biomass)

# Train biomass estimation on all 5 folds of previously created splits.
for fold in range(5):
    !python scripts/02_train.py \
        --data_folder "../0_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path "data/processed/ept-14/01_EPT+1_processed_5splits_Species_DNA.csv" \
        --label "Log+1_weights" \
        --fold {fold} \
        --class_map 'none' \
        --imsize 224 \
        --batch_size 256 \
        --aug 'EPT-biomass' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 100 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'l1' \
        --lr 0.00020893 \
        --auto_lr 'False' \
        --log_dir '+1_EPT-14-biomass' \
        --out_folder 'outputs' \
        --out_prefix '+1_EPT-14-biomass' \
        --deterministic 'True' \
        --precision '16-mixed'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Seed set to 42
None
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in ./wandb/run-20241025_121919-241025-1219-9017
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run +1_EPT-14-biomass_efficientnet_b0_f0_241025-1219-9017
wandb: ⭐️ View project at https://wandb.ai/uni-due/%2B1_EPT-14-biomass
wandb: 🚀 View

Epoch 2: 100%|████████████████████| 355/355 [02:54<00:00,  2.03it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 355/355 [02:55<00:00,  2.02it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  83%|██████████████▉   | 74/89 [00:20<00:04,  3.63it/s]


Epoch 4: 100%|████████████████████| 355/355 [02:54<00:00,  2.03it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|████████████████████| 355/355 [02:56<00:00,  2.02it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  92%|████████████████▌ | 82/89 [00:22<00:01,  3.66it/s]


Epoch 7: 100%|████████████████████| 355/355 [02:56<00:00,  2.02it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|████████████████████| 355/355 [02:55<00:00,  2.02it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|████████████████████| 355/355 [02:54<00:00,  2.04it/s, v_num=9017]


Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 10: 100%|███████████████████| 355/355 [02:56<00:00,  2.01it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 11: 100%|███████████████████| 355/355 [02:57<00:00,  2.00it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   6%|█                  | 5/89 [00:00<00:08,  9.60it/s]


...

Epoch 30: 100%|███████████████████| 355/355 [02:56<00:00,  2.01it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 31: 100%|███████████████████| 355/355 [02:54<00:00,  2.03it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  22%|████              | 20/89 [00:04<00:15,  4.42it/s]


Epoch 32: 100%|███████████████████| 355/355 [02:54<00:00,  2.04it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 33: 100%|███████████████████| 355/355 [02:54<00:00,  2.04it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  31%|█████▋            | 28/89 [00:08<00:17,  3.50it/s]


Epoch 34: 100%|███████████████████| 355/355 [02:55<00:00,  2.03it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 35: 100%|███████████████████| 355/355 [02:54<00:00,  2.03it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  40%|███████▎          | 36/89 [00:09<00:14,  3.72it/s]


Epoch 36: 100%|███████████████████| 355/355 [02:54<00:00,  2.04it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 37: 100%|███████████████████| 355/355 [02:53<00:00,  2.04it/s, v_num=9017]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  49%|████████▉         | 44/89 [00:12<00:13,  3.42it/s]


Validation DataLoader 0:  99%|█████████████████▊| 88/89 [00:24<00:00,  3.65it/s]

In [2]:
import os
import glob

def list_files_with_prefix(directory, prefix):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if not f.endswith('_last.ckpt')]     # Filter out files ending with '_last.ckpt'
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    for subfolder in subfolders:
        fold_dict[int(subfolder[-1])].extend(list_ckpts(subfolder))
        
directory = 'outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0'
prefix = ''

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files
setting_folders = list_files_with_prefix(directory, prefix)

extend_fold_dict(directory)

In [3]:
fold_dict

{0: ['outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f0/+1_EPT-14-biomass_efficientnet_b0_f0_241025-1219-9017_epoch82_val-loss0.17.ckpt'],
 1: ['outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f1/+1_EPT-14-biomass_efficientnet_b0_f1_241025-1759-72f9_epoch00_val-loss0.23.ckpt'],
 2: ['outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f2/+1_EPT-14-biomass_efficientnet_b0_f2_241025-2356-6e57_epoch01_val-loss0.18.ckpt'],
 3: ['outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f3/+1_EPT-14-biomass_efficientnet_b0_f3_241026-0534-612c_epoch37_val-loss0.22.ckpt'],
 4: ['outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f4/+1_EPT-14-biomass_efficientnet_b0_f4_241026-1120-f81d_epoch25_val-loss0.25.ckpt']}

In [5]:
for fold, ls in fold_dict.items():
    for file in ls:
        # Automatic predictiongs using the first (best) model checkpoint in the directory.
        !python scripts/03_predict.py \
            --data_folder "../0_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/ept-14/01_EPT+1_processed_5splits_Species_DNA.csv" \
            --label "Log+1_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 1024 \
            --aug 'EPT-biomass' \
            --out_folder 'outputs' \
            --tta 'False' \
            --out_prefix '+1_EPT-14-biomass' \
            --ckpt_path {file}

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing DataLoader 0: 100%|█████████████████████| 28/28 [00:49<00:00,  0.56it/s]
────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
     test/loss_epoch        0.2681639492511749
────────────────────────────────────────────────────────────────────────────────
outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f0/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f0_241025-1219-9017_epoch82_val-loss0.17_EPT-biomass.csv
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing DataLoader 0: 100%|█████████████████████| 28/28 [00:29<00:00,  0.96it/s]
────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
     test/loss_epoch        0.19697776436805725
────────────────────────────────────────────────────────────────────────────────
outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f4/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f4_241026-1120-f81d_epoch25_val-loss0.25_EPT-biomass.csv


In [ ]:
## Until here, the code works fine. The following parts are still more or less experimental and for sure not cleaned up.

In [4]:
import os
import glob

def list_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, f"{prefix}*")
    files = glob.glob(pattern)
    
    return files

def list_csvs(directory): 
    pattern = os.path.join(directory, 'predictions', 'biodiscover_none', '*.csv')
    csv_files = glob.glob(pattern)
    return csv_files

def extend_run_dict_csv(directory):
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    for subfolder in subfolders:
        run_dict_csv[subfolder].extend(list_csvs(subfolder))

run_dict_csv={}
for run in list_files_with_prefix(directory, prefix):
    run_dict_csv[run] = []
# List the files
setting_folders = list_files_with_prefix(directory, prefix)

extend_run_dict_csv(directory)

In [6]:
import pandas as pd
from pathlib import Path

df_f0 = pd.read_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f0/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f0_241025-1219-9017_epoch82_val-loss0.17_EPT-biomass.csv')
df_f1 = pd.read_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f1/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f1_241025-1759-72f9_epoch00_val-loss0.23_EPT-biomass.csv')
df_f2 = pd.read_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f2/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f2_241025-2356-6e57_epoch01_val-loss0.18_EPT-biomass.csv')
df_f3 = pd.read_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f3/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f3_241026-0534-612c_epoch37_val-loss0.22_EPT-biomass.csv')
df_f4 = pd.read_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/f4/predictions/biodiscover_EPT-biomass/+1_EPT-14-biomass_+1_EPT-14-biomass_efficientnet_b0_f4_241026-1120-f81d_epoch25_val-loss0.25_EPT-biomass.csv')
ref = pd.read_csv("data/processed/ept-14/01_EPT+1_processed_5splits_Species_DNA.csv")

df = pd.concat([df_f0, df_f1, df_f2, df_f3, df_f4])

# Get just the filename from the full path
df["fname"] = df["fname"].apply(lambda x: Path(x).name)
preds = df[["fname", "y_pred"]].set_index("fname")

# Join the dataframes. 'inner' to keep only the rows that are in both dataframes
results_df = ref.join(preds, how="inner", on="Image File Name")  

# Choose wanted columns
results_df = results_df[['Log+1_weights', "y_pred", "individual"]]

# Grouped by individual
results_df_grouped = results_df.groupby(["individual"]).apply(lambda group: group[(group >= group.quantile(0.05)) & (group <= group.quantile(0.95))].mean())

results_df_grouped.to_csv('outputs/biodiscover/+1_EPT-14-biomass_efficientnet_b0/+1_EPT-14_processed_5splits_Species_DNA_grouped_predictions_mean.csv')


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
def confusion_matrixX(y_true, 
                      y_pred, 
                      classes, 
                      figsize=(15,15), 
                      fonts=(18,10,18), 
                      rotate=False,
                      grid=False,
                      normalize='true'):

    # Check the inputs
    s_true = set(np.unique(y_true))
    s_pred = set(np.unique(y_pred))

    if classes == 'union':
        classes = sorted(list(s_true | s_pred))

    if len((s_true | s_pred) - set(classes)) > 0:
        warnings.warn(f"The inputs contain classes not present in the class list: {(s_true | s_pred) - set(classes)}")

    cm = confusion_matrix(y_true, 
                          y_pred, 
                          labels=classes,
                          normalize=normalize)
    
    if normalize:
        cm = cm*100
    fig, ax = plt.subplots(figsize=figsize)

    mask = np.zeros_like(cm)
    mask[cm==0] = 1
    sns.heatmap(cm, 
                annot=True, 
                annot_kws={"size": fonts[0]},
                fmt='.0f', 
                cmap="YlGnBu",
                xticklabels=classes, 
                yticklabels=classes,
                mask=mask,
                square=True, 
                cbar=False,
                ax=ax)
    
    if grid:
        ax.set_axisbelow(True)
        ax.yaxis.grid(color='gray', linestyle='dashed')
        ax.xaxis.grid(color='gray', linestyle='dashed')

    ax.set_ylabel('True label', size=fonts[2])
    ax.set_xlabel('Predicted label', size=fonts[2])
    ax.yaxis.set_ticks_position('none') 
    ax.xaxis.set_ticks_position('none') 
    
    if rotate:
        if isinstance(rotate, list):
            rot_x = rotate[0]
            rot_y = rotate[1]
        else:
            rot_x = rot_y = rotate
        plt.setp(ax.get_yticklabels(), rotation=rot_x, ha="right", rotation_mode="anchor", size=fonts[1])
        plt.setp(ax.get_xticklabels(), rotation=rot_y, ha="right", rotation_mode="anchor", size=fonts[1])
    else:
        plt.setp(ax.get_yticklabels(), size=fonts[1])
        plt.setp(ax.get_xticklabels(), size=fonts[1])
    plt.show()
        
from sklearn.metrics import classification_report
def classification_reportX(*args, 
                           figsize=None, 
                           fonts=(12,10,10),
                           rotate=True, 
                           bbox_anchor=(1.15,0.8), 
                           **kwargs):
    """Extends classification report by adding an useful plot for the performance across classes"""
    r = classification_report(*args, **kwargs, output_dict=True)
    rdf0 = pd.DataFrame(r)
    rdf = rdf0.T.iloc[:-3,:].sort_values('support',ascending=False)
    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()

    xbar = range(len(rdf))
    ax1.bar(xbar,rdf['support'], alpha=0.2)
    ax2.plot(rdf['f1-score'], 'ro', label='f1-score')
    ax2.plot(rdf['precision'], 'g*', label='precision', alpha=0.5)
    ax2.plot(rdf['recall'], 'b*', label='recall', alpha=0.5)

    ax2.vlines(xbar, rdf['f1-score'], rdf['precision'], 'g')
    ax2.vlines(xbar, rdf['f1-score'], rdf['recall'], 'b')

    ax2.hlines(rdf0['weighted avg']['f1-score'], 0,len(rdf), color='r', linestyle='--', label='weighted f1-score')
    ax2.hlines(rdf0['macro avg']['f1-score'], 0,len(rdf), color='r', linestyle='-.', label='macro f1-score')
    ax2.hlines(rdf0['accuracy']['recall'], 0,len(rdf), color='b', linestyle='--', label='accuracy')
    
    if rotate:
        plt.setp(ax1.get_xticklabels(), 
                 rotation=45, 
                 ha="right", 
                 rotation_mode="anchor",
                 size=fonts[1])

    fig.legend(bbox_to_anchor=bbox_anchor,
               prop={"size": fonts[2]})
    
    ax2.set_ylabel("Score", size=fonts[0])
    ax1.set_ylabel("Support", size=fonts[0])
    plt.setp(ax1.get_yticklabels(),
             size=fonts[0])
    plt.setp(ax2.get_yticklabels(),
             size=fonts[0])
    
    return classification_report(*args, **kwargs)

In [ ]:
import pandas as pd
confusion_df = pd.read_csv('outputs/biodiscover/02_ept-14-biomass_efficientnet_b0/01_EPT_processed_5splits_Species_DNA_grouped_predictions.csv')

confusion_matrixX(confusion_df['y_true'], confusion_df['y_pred'], 'union')